In [1]:
import ee
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
import os
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from IPython.display import Image
import json
import time

import folium
import cv2

In [3]:
driver=webdriver.Firefox(executable_path=".\geckodriver.exe")
driver.set_window_position(0, 0)
driver.set_window_size(1024, 768)

C:\Users\Nannu\AppData\Local\Temp/ipykernel_15740/2332002925.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Firefox(executable_path=".\geckodriver.exe")


In [4]:
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [5]:
def get_s2_sr_cld_col(aoi, start_date, end_date,cloud_filter):
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', cloud_filter)))
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    not_water = img.select('SCL').neq(6)
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    img_cloud = add_cloud_bands(img)
    img_cloud_shadow = add_shadow_bands(img_cloud)
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))
    return img_cloud_shadow.addBands(is_cld_shdw)

def add_ee_layer(self, ee_image_object, vis_params, name, show=True, opacity=1, min_zoom=0):
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        show=show,
        opacity=opacity,
        min_zoom=min_zoom,
        overlay=True,
        control=True
        ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [6]:
def display_cloud_layers(AOI,col,verbose=False):
    img = col.mosaic()
    clouds = img.select('clouds').selfMask()
    shadows = img.select('shadows').selfMask()
    dark_pixels = img.select('dark_pixels').selfMask()
    probability = img.select('probability')
    cloudmask = img.select('cloudmask').selfMask()
    cloud_transform = img.select('cloud_transform')

    center = AOI.centroid(10).coordinates().reverse().getInfo()
    m = folium.Map(location=center, zoom_start=12)

    m.add_ee_layer(img,
                   {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
                   'S2 image', True, 1, 9)
    m.add_ee_layer(probability,
                   {'min': 0, 'max': 100},
                   'probability (cloud)', False, 1, 9)
    m.add_ee_layer(clouds,
                   {'palette': 'e056fd'},
                   'clouds', False, 1, 9)
    m.add_ee_layer(cloud_transform,
                   {'min': 0, 'max': 1, 'palette': ['white', 'black']},
                   'cloud_transform', False, 1, 9)
    m.add_ee_layer(dark_pixels,
                   {'palette': 'orange'},
                   'dark_pixels', False, 1, 9)
    m.add_ee_layer(shadows, {'palette': 'yellow'},
                   'shadows', False, 1, 9)
    # m.add_ee_layer(cloudmask, {'palette': 'orange'},
    #                'cloudmask', True, 0.5, 9)

    m.add_child(folium.LayerControl())
    if verbose:
        display(m)
    return m

def crop_img(fileName):
    pngName1="./uncropedpngFiles/"+fileName+".png"
    pngName2="./cropedpngFiles/"+fileName+".png"
    img = cv2.imread(pngName1)
    img1=img[0:740,60:960]
    cv2.imwrite(pngName2,img1)

def map_to_png(mapObj, fileName):
    htmlFile="./htmlFiles/"+fileName+".html"
    mapObj.save(htmlFile)
    driver.get('file://{0}/{1}'.format(os.getcwd(), htmlFile))
    time.sleep(5)
    pngName="./uncropedpngFiles/"+fileName+".png"
    driver.save_screenshot(pngName)
    # driver.quit()
    crop_img(fileName)


In [7]:
cloud_filters=[1,60]
dates=[
    [['2018-01-01','2018-03-28'],['2018-04-01','2018-06-28'],['2018-07-01','2018-09-28'],['2018-10-01','2018-12-28']],
    [['2019-01-01','2019-03-28'],['2019-04-01','2019-06-28'],['2019-07-01','2019-09-28'],['2019-10-01','2019-12-28']],
    [['2020-01-01','2020-03-28'],['2020-04-01','2020-06-28'],['2020-07-01','2020-09-28'],['2020-10-01','2020-12-28']],
    [['2021-01-01','2021-03-28'],['2021-04-01','2021-06-28'],['2021-07-01','2021-09-28'],['2021-10-01','2021-12-28']]
]
aois={"0": [[75.8979780320574, 30.24043937830989], [75.8979780320574, 30.231429415897423], [75.91456738752615, 30.231429415897423], [75.91456738752615, 30.24043937830989]], "1": [[75.91456738752615, 30.24043937830989], [75.91456738752615, 30.231429415897423], [75.9311567429949, 30.231429415897423], [75.9311567429949, 30.24043937830989]], "2": [[75.9311567429949, 30.24043937830989], [75.9311567429949, 30.231429415897423], [75.94774609846365, 30.231429415897423], [75.94774609846365, 30.24043937830989]], "3": [[75.94774609846365, 30.24043937830989], [75.94774609846365, 30.231429415897423], [75.9643354539324, 30.231429415897423], [75.9643354539324, 30.24043937830989]], "4": [[75.9643354539324, 30.24043937830989], [75.9643354539324, 30.231429415897423], [75.98092480940116, 30.231429415897423], [75.98092480940116, 30.24043937830989]], "5": [[75.98092480940116, 30.24043937830989], [75.98092480940116, 30.231429415897423], [75.99751416486991, 30.231429415897423], [75.99751416486991, 30.24043937830989]], "6": [[75.99751416486991, 30.24043937830989], [75.99751416486991, 30.231429415897423], [76.01410352033867, 30.231429415897423], [76.01410352033867, 30.24043937830989]], "7": [[76.01410352033867, 30.24043937830989], [76.01410352033867, 30.231429415897423], [76.03069287580742, 30.231429415897423], [76.03069287580742, 30.24043937830989]], "8": [[76.03069287580742, 30.24043937830989], [76.03069287580742, 30.231429415897423], [76.04728223127617, 30.231429415897423], [76.04728223127617, 30.24043937830989]], "9": [[76.04728223127617, 30.24043937830989], [76.04728223127617, 30.231429415897423], [76.06387158674492, 30.231429415897423], [76.06387158674492, 30.24043937830989]], "10": [[76.06387158674492, 30.24043937830989], [76.06387158674492, 30.231429415897423], [76.08046094221368, 30.231429415897423], [76.08046094221368, 30.24043937830989]], "11": [[76.08046094221368, 30.24043937830989], [76.08046094221368, 30.231429415897423], [76.09705029768243, 30.231429415897423], [76.09705029768243, 30.24043937830989]], "12": [[76.09705029768243, 30.24043937830989], [76.09705029768243, 30.231429415897423], [76.11363965315118, 30.231429415897423], [76.11363965315118, 30.24043937830989]], "13": [[76.11363965315118, 30.24043937830989], [76.11363965315118, 30.231429415897423], [76.13022900861993, 30.231429415897423], [76.13022900861993, 30.24043937830989]], "14": [[76.13022900861993, 30.24043937830989], [76.13022900861993, 30.231429415897423], [76.14681836408869, 30.231429415897423], [76.14681836408869, 30.24043937830989]], "15": [[76.14681836408869, 30.24043937830989], [76.14681836408869, 30.231429415897423], [76.16340771955744, 30.231429415897423], [76.16340771955744, 30.24043937830989]], "16": [[76.16340771955744, 30.24043937830989], [76.16340771955744, 30.231429415897423], [76.1799970750262, 30.231429415897423], [76.1799970750262, 30.24043937830989]], "17": [[76.1799970750262, 30.24043937830989], [76.1799970750262, 30.231429415897423], [76.19658643049495, 30.231429415897423], [76.19658643049495, 30.24043937830989]], "18": [[76.19658643049495, 30.24043937830989], [76.19658643049495, 30.231429415897423], [76.2131757859637, 30.231429415897423], [76.2131757859637, 30.24043937830989]], "19": [[76.2131757859637, 30.24043937830989], [76.2131757859637, 30.231429415897423], [76.22976514143245, 30.231429415897423], [76.22976514143245, 30.24043937830989]], "20": [[76.22976514143245, 30.24043937830989], [76.22976514143245, 30.231429415897423], [76.2463544969012, 30.231429415897423], [76.2463544969012, 30.24043937830989]], "21": [[76.2463544969012, 30.24043937830989], [76.2463544969012, 30.231429415897423], [76.26294385236996, 30.231429415897423], [76.26294385236996, 30.24043937830989]], "22": [[76.26294385236996, 30.24043937830989], [76.26294385236996, 30.231429415897423], [76.27953320783871, 30.231429415897423], [76.27953320783871, 30.24043937830989]], "23": [[76.27953320783871, 30.24043937830989], [76.27953320783871, 30.231429415897423], [76.29612256330746, 30.231429415897423], [76.29612256330746, 30.24043937830989]], "24": [[76.29612256330746, 30.24043937830989], [76.29612256330746, 30.231429415897423], [76.31271191877622, 30.231429415897423], [76.31271191877622, 30.24043937830989]], "25": [[76.31271191877622, 30.24043937830989], [76.31271191877622, 30.231429415897423], [76.32930127424497, 30.231429415897423], [76.32930127424497, 30.24043937830989]], "26": [[76.32930127424497, 30.24043937830989], [76.32930127424497, 30.231429415897423], [76.34589062971372, 30.231429415897423], [76.34589062971372, 30.24043937830989]], "27": [[76.34589062971372, 30.24043937830989], [76.34589062971372, 30.231429415897423], [76.36247998518247, 30.231429415897423], [76.36247998518247, 30.24043937830989]], "28": [[76.36247998518247, 30.24043937830989], [76.36247998518247, 30.231429415897423], [76.37906934065123, 30.231429415897423], [76.37906934065123, 30.24043937830989]], "29": [[76.37906934065123, 30.24043937830989], [76.37906934065123, 30.231429415897423], [76.39565869611998, 30.231429415897423], [76.39565869611998, 30.24043937830989]], "30": [[76.39565869611998, 30.24043937830989], [76.39565869611998, 30.231429415897423], [76.41224805158873, 30.231429415897423], [76.41224805158873, 30.24043937830989]], "31": [[76.41224805158873, 30.24043937830989], [76.41224805158873, 30.231429415897423], [76.42883740705749, 30.231429415897423], [76.42883740705749, 30.24043937830989]], "32": [[76.42883740705749, 30.24043937830989], [76.42883740705749, 30.231429415897423], [76.44542676252624, 30.231429415897423], [76.44542676252624, 30.24043937830989]], "33": [[76.44542676252624, 30.24043937830989], [76.44542676252624, 30.231429415897423], [76.46201611799499, 30.231429415897423], [76.46201611799499, 30.24043937830989]], "34": [[76.46201611799499, 30.24043937830989], [76.46201611799499, 30.231429415897423], [76.47860547346374, 30.231429415897423], [76.47860547346374, 30.24043937830989]], "35": [[76.47860547346374, 30.24043937830989], [76.47860547346374, 30.231429415897423], [76.4951948289325, 30.231429415897423], [76.4951948289325, 30.24043937830989]], "36": [[76.4951948289325, 30.24043937830989], [76.4951948289325, 30.231429415897423], [76.51178418440125, 30.231429415897423], [76.51178418440125, 30.24043937830989]], "37": [[76.51178418440125, 30.24043937830989], [76.51178418440125, 30.231429415897423], [76.52837353987, 30.231429415897423], [76.52837353987, 30.24043937830989]], "38": [[76.52837353987, 30.24043937830989], [76.52837353987, 30.231429415897423], [76.54496289533876, 30.231429415897423], [76.54496289533876, 30.24043937830989]], "39": [[76.54496289533876, 30.24043937830989], [76.54496289533876, 30.231429415897423], [76.56155225080751, 30.231429415897423], [76.56155225080751, 30.24043937830989]], "40": [[76.56155225080751, 30.24043937830989], [76.56155225080751, 30.231429415897423], [76.57814160627626, 30.231429415897423], [76.57814160627626, 30.24043937830989]], "41": [[76.57814160627626, 30.24043937830989], [76.57814160627626, 30.231429415897423], [76.59473096174501, 30.231429415897423], [76.59473096174501, 30.24043937830989]], "42": [[76.59473096174501, 30.24043937830989], [76.59473096174501, 30.231429415897423], [76.61132031721377, 30.231429415897423], [76.61132031721377, 30.24043937830989]], "43": [[76.61132031721377, 30.24043937830989], [76.61132031721377, 30.231429415897423], [76.62790967268252, 30.231429415897423], [76.62790967268252, 30.24043937830989]], "44": [[76.62790967268252, 30.24043937830989], [76.62790967268252, 30.231429415897423], [76.64449902815127, 30.231429415897423], [76.64449902815127, 30.24043937830989]], "45": [[76.64449902815127, 30.24043937830989], [76.64449902815127, 30.231429415897423], [76.66108838362003, 30.231429415897423], [76.66108838362003, 30.24043937830989]], "46": [[76.66108838362003, 30.24043937830989], [76.66108838362003, 30.231429415897423], [76.67767773908878, 30.231429415897423], [76.67767773908878, 30.24043937830989]], "47": [[76.67767773908878, 30.24043937830989], [76.67767773908878, 30.231429415897423], [76.69426709455753, 30.231429415897423], [76.69426709455753, 30.24043937830989]], "48": [[76.69426709455753, 30.24043937830989], [76.69426709455753, 30.231429415897423], [76.71085645002628, 30.231429415897423], [76.71085645002628, 30.24043937830989]], "49": [[76.71085645002628, 30.24043937830989], [76.71085645002628, 30.231429415897423], [76.72744580549504, 30.231429415897423], [76.72744580549504, 30.24043937830989]], "50": [[76.72744580549504, 30.24043937830989], [76.72744580549504, 30.231429415897423], [76.74403516096379, 30.231429415897423], [76.74403516096379, 30.24043937830989]], "51": [[76.74403516096379, 30.24043937830989], [76.74403516096379, 30.231429415897423], [76.76062451643254, 30.231429415897423], [76.76062451643254, 30.24043937830989]], "52": [[76.76062451643254, 30.24043937830989], [76.76062451643254, 30.231429415897423], [76.7772138719013, 30.231429415897423], [76.7772138719013, 30.24043937830989]], "53": [[76.7772138719013, 30.24043937830989], [76.7772138719013, 30.231429415897423], [76.79380322737005, 30.231429415897423], [76.79380322737005, 30.24043937830989]], "54": [[76.79380322737005, 30.24043937830989], [76.79380322737005, 30.231429415897423], [76.8103925828388, 30.231429415897423], [76.8103925828388, 30.24043937830989]], "55": [[76.8103925828388, 30.24043937830989], [76.8103925828388, 30.231429415897423], [76.82698193830755, 30.231429415897423], [76.82698193830755, 30.24043937830989]], "56": [[76.82698193830755, 30.24043937830989], [76.82698193830755, 30.231429415897423], [76.84357129377631, 30.231429415897423], [76.84357129377631, 30.24043937830989]], "57": [[76.84357129377631, 30.24043937830989], [76.84357129377631, 30.231429415897423], [76.86016064924506, 30.231429415897423], [76.86016064924506, 30.24043937830989]], "58": [[76.86016064924506, 30.24043937830989], [76.86016064924506, 30.231429415897423], [76.87675000471381, 30.231429415897423], [76.87675000471381, 30.24043937830989]], "59": [[76.87675000471381, 30.24043937830989], [76.87675000471381, 30.231429415897423], [76.89333936018257, 30.231429415897423], [76.89333936018257, 30.24043937830989]], "60": [[76.89333936018257, 30.24043937830989], [76.89333936018257, 30.231429415897423], [76.90992871565132, 30.231429415897423], [76.90992871565132, 30.24043937830989]], "61": [[76.90992871565132, 30.24043937830989], [76.90992871565132, 30.231429415897423], [76.92651807112007, 30.231429415897423], [76.92651807112007, 30.24043937830989]], "62": [[76.92651807112007, 30.24043937830989], [76.92651807112007, 30.231429415897423], [76.94310742658882, 30.231429415897423], [76.94310742658882, 30.24043937830989]], "63": [[76.94310742658882, 30.24043937830989], [76.94310742658882, 30.231429415897423], [76.95969678205758, 30.231429415897423], [76.95969678205758, 30.24043937830989]], "64": [[76.95969678205758, 30.24043937830989], [76.95969678205758, 30.231429415897423], [76.97628613752633, 30.231429415897423], [76.97628613752633, 30.24043937830989]], "65": [[76.97628613752633, 30.24043937830989], [76.97628613752633, 30.231429415897423], [76.99287549299508, 30.231429415897423], [76.99287549299508, 30.24043937830989]], "66": [[76.99287549299508, 30.24043937830989], [76.99287549299508, 30.231429415897423], [77.00946484846384, 30.231429415897423], [77.00946484846384, 30.24043937830989]], "67": [[77.00946484846384, 30.24043937830989], [77.00946484846384, 30.231429415897423], [77.02605420393259, 30.231429415897423], [77.02605420393259, 30.24043937830989]], "68": [[77.02605420393259, 30.24043937830989], [77.02605420393259, 30.231429415897423], [77.04264355940134, 30.231429415897423], [77.04264355940134, 30.24043937830989]], "69": [[77.04264355940134, 30.24043937830989], [77.04264355940134, 30.231429415897423], [77.0592329148701, 30.231429415897423], [77.0592329148701, 30.24043937830989]], "70": [[77.0592329148701, 30.24043937830989], [77.0592329148701, 30.231429415897423], [77.07582227033885, 30.231429415897423], [77.07582227033885, 30.24043937830989]], "71": [[77.07582227033885, 30.24043937830989], [77.07582227033885, 30.231429415897423], [77.0924116258076, 30.231429415897423], [77.0924116258076, 30.24043937830989]], "72": [[77.0924116258076, 30.24043937830989], [77.0924116258076, 30.231429415897423], [77.10900098127635, 30.231429415897423], [77.10900098127635, 30.24043937830989]], "73": [[77.10900098127635, 30.24043937830989], [77.10900098127635, 30.231429415897423], [77.1255903367451, 30.231429415897423], [77.1255903367451, 30.24043937830989]], "74": [[77.1255903367451, 30.24043937830989], [77.1255903367451, 30.231429415897423], [77.14217969221386, 30.231429415897423], [77.14217969221386, 30.24043937830989]], "75": [[77.14217969221386, 30.24043937830989], [77.14217969221386, 30.231429415897423], [77.15876904768261, 30.231429415897423], [77.15876904768261, 30.24043937830989]], "76": [[77.15876904768261, 30.24043937830989], [77.15876904768261, 30.231429415897423], [77.17535840315136, 30.231429415897423], [77.17535840315136, 30.24043937830989]], "77": [[77.17535840315136, 30.24043937830989], [77.17535840315136, 30.231429415897423], [77.19194775862012, 30.231429415897423], [77.19194775862012, 30.24043937830989]], "78": [[77.19194775862012, 30.24043937830989], [77.19194775862012, 30.231429415897423], [77.20853711408887, 30.231429415897423], [77.20853711408887, 30.24043937830989]], "79": [[77.20853711408887, 30.24043937830989], [77.20853711408887, 30.231429415897423], [77.22512646955762, 30.231429415897423], [77.22512646955762, 30.24043937830989]], "80": [[77.22512646955762, 30.24043937830989], [77.22512646955762, 30.231429415897423], [77.24171582502638, 30.231429415897423], [77.24171582502638, 30.24043937830989]], "81": [[77.24171582502638, 30.24043937830989], [77.24171582502638, 30.231429415897423], [77.25830518049513, 30.231429415897423], [77.25830518049513, 30.24043937830989]], "82": [[77.25830518049513, 30.24043937830989], [77.25830518049513, 30.231429415897423], [77.27489453596388, 30.231429415897423], [77.27489453596388, 30.24043937830989]], "83": [[77.27489453596388, 30.24043937830989], [77.27489453596388, 30.231429415897423], [77.29148389143263, 30.231429415897423], [77.29148389143263, 30.24043937830989]], "84": [[77.29148389143263, 30.24043937830989], [77.29148389143263, 30.231429415897423], [77.30807324690139, 30.231429415897423], [77.30807324690139, 30.24043937830989]], "85": [[77.30807324690139, 30.24043937830989], [77.30807324690139, 30.231429415897423], [77.32466260237014, 30.231429415897423], [77.32466260237014, 30.24043937830989]], "86": [[77.32466260237014, 30.24043937830989], [77.32466260237014, 30.231429415897423], [77.34125195783889, 30.231429415897423], [77.34125195783889, 30.24043937830989]], "87": [[77.34125195783889, 30.24043937830989], [77.34125195783889, 30.231429415897423], [77.35784131330765, 30.231429415897423], [77.35784131330765, 30.24043937830989]], "88": [[77.35784131330765, 30.24043937830989], [77.35784131330765, 30.231429415897423], [77.3744306687764, 30.231429415897423], [77.3744306687764, 30.24043937830989]], "89": [[77.3744306687764, 30.24043937830989], [77.3744306687764, 30.231429415897423], [77.39102002424515, 30.231429415897423], [77.39102002424515, 30.24043937830989]], "90": [[77.39102002424515, 30.24043937830989], [77.39102002424515, 30.231429415897423], [77.4076093797139, 30.231429415897423], [77.4076093797139, 30.24043937830989]], "91": [[77.4076093797139, 30.24043937830989], [77.4076093797139, 30.231429415897423], [77.42419873518266, 30.231429415897423], [77.42419873518266, 30.24043937830989]], "92": [[77.42419873518266, 30.24043937830989], [77.42419873518266, 30.231429415897423], [77.44078809065141, 30.231429415897423], [77.44078809065141, 30.24043937830989]], "93": [[77.44078809065141, 30.24043937830989], [77.44078809065141, 30.231429415897423], [77.45737744612016, 30.231429415897423], [77.45737744612016, 30.24043937830989]], "94": [[77.45737744612016, 30.24043937830989], [77.45737744612016, 30.231429415897423], [77.47396680158892, 30.231429415897423], [77.47396680158892, 30.24043937830989]], "95": [[77.47396680158892, 30.24043937830989], [77.47396680158892, 30.231429415897423], [77.49055615705767, 30.231429415897423], [77.49055615705767, 30.24043937830989]], "96": [[77.49055615705767, 30.24043937830989], [77.49055615705767, 30.231429415897423], [77.50714551252642, 30.231429415897423], [77.50714551252642, 30.24043937830989]], "97": [[77.50714551252642, 30.24043937830989], [77.50714551252642, 30.231429415897423], [77.52373486799517, 30.231429415897423], [77.52373486799517, 30.24043937830989]], "98": [[77.52373486799517, 30.24043937830989], [77.52373486799517, 30.231429415897423], [77.54032422346393, 30.231429415897423], [77.54032422346393, 30.24043937830989]], "99": [[77.54032422346393, 30.24043937830989], [77.54032422346393, 30.231429415897423], [77.55691357893268, 30.231429415897423], [77.55691357893268, 30.24043937830989]]}

In [8]:
fp1=open("./logFiles/CF1.txt","w")
fp2=open("./logFiles/CF60.txt","w")

for id in aois:
    print(">> " +str(id) + " started")
    for x in cloud_filters:
        for y in dates:
            for z in y:
                CLOUD_FILTER=x
                START_DATE=z[0]
                END_DATE=z[1]
                AOI = ee.Geometry.Polygon(aois[id])
                fileName = str(id)+'_CF'+str(CLOUD_FILTER)+'_SD'+str(START_DATE).replace('-',"_")+'_ED'+str(END_DATE).replace('-',"_")
                try:
                    s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE,CLOUD_FILTER)
                    s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
                    m = display_cloud_layers(AOI,s2_sr_cld_col_eval_disp)
                    map_to_png(m, fileName)
                    if CLOUD_FILTER==1:
                        print(fileName,file=fp1)
                    else:
                        print(fileName,file=fp2)
                except:
                    if CLOUD_FILTER==1:
                        print(fileName + " failed",file=fp1)
                    else:
                        print(fileName + " failed",file=fp2)
    print(">> "+str(id) + " ended")


fp1.close()
fp2.close()

>> 0 started
>> 0 ended
>> 1 started
>> 1 ended
>> 2 started
>> 2 ended
>> 3 started
>> 3 ended
>> 4 started
>> 4 ended
>> 5 started
>> 5 ended
>> 6 started
>> 6 ended
>> 7 started
>> 7 ended
>> 8 started
>> 8 ended
>> 9 started
>> 9 ended
>> 10 started
>> 10 ended
>> 11 started
>> 11 ended
>> 12 started
>> 12 ended
>> 13 started
>> 13 ended
>> 14 started
>> 14 ended
>> 15 started
>> 15 ended
>> 16 started
>> 16 ended
>> 17 started
>> 17 ended
>> 18 started
>> 18 ended
>> 19 started
>> 19 ended
>> 20 started
>> 20 ended
>> 21 started
>> 21 ended
>> 22 started
>> 22 ended
>> 23 started
>> 23 ended
>> 24 started
>> 24 ended
>> 25 started
>> 25 ended
>> 26 started
>> 26 ended
>> 27 started
>> 27 ended
>> 28 started
>> 28 ended
>> 29 started
>> 29 ended
>> 30 started
>> 30 ended
>> 31 started
>> 31 ended
>> 32 started
>> 32 ended
>> 33 started
>> 33 ended
>> 34 started
>> 34 ended
>> 35 started
>> 35 ended
>> 36 started
>> 36 ended
>> 37 started
>> 37 ended
>> 38 started
>> 38 ended
>> 39 

In [11]:
driver.quit()
# fileName="2"
# CLOUD_FILTER = 100
# AOI = ee.Geometry.Polygon(
#         [[[75.8979780320574, 30.24043937830989],
#         [75.8979780320574, 30.231429415897423],
#         [75.91456738752615, 30.231429415897423],
#         [75.91456738752615, 30.24043937830989]]])
# START_DATE = '2020-06-01'
# END_DATE = '2020-06-02'


# s2_sr_cld_col_eval = get_s2_sr_cld_col(AOI, START_DATE, END_DATE,CLOUD_FILTER)
# s2_sr_cld_col_eval_disp = s2_sr_cld_col_eval.map(add_cld_shdw_mask)
# m = display_cloud_layers(AOI,s2_sr_cld_col_eval_disp,True)
# map_to_png(m, fileName)